In [1]:
import sqlite3
import pandas as pd
import movingpandas as mpd
from datetime import timedelta
import numpy as np

In [2]:
activity_records_connection = sqlite3.connect('/Users/dominicdowding/HealthData/DBs/garmin_activities.db')
activity_records_df = pd.read_sql_query("SELECT a.activity_id, a.start_lat, a.start_long, a.stop_lat, a.stop_long, a.name, a.type, a.start_time, a.stop_time, a.moving_time, a.sport, a.distance, a.cycles, a.laps, a.avg_hr, a.max_hr, a.calories, a.avg_cadence, a.max_cadence, a.avg_speed, a.max_speed, a.ascent, a.max_temperature, a.min_temperature, ar.record, ar.timestamp, ar.position_lat, ar.position_long, ar.distance as activity_distance, ar.cadence, ar.hr, ar.altitude, ar.speed, ar.temperature FROM  activities a JOIN activity_records ar on a.activity_id = ar.activity_id", activity_records_connection)
activity_records_df['day'] = activity_records_df['timestamp'].str[:10]

OperationalError: unable to open database file

In [ ]:
# Metrics to find
# Pace e.g distance covered per minute e.g 7 minute mile pace
# Heart rate zone e.g 60-70% of max heart rate zone

summary_records_connection = sqlite3.connect('/Users/dominicdowding/HealthData/DBs/summary.db')
summary_records_df = pd.read_sql_query("SELECT * FROM days_summary", summary_records_connection)
summary_records_df = summary_records_df.replace(to_replace='None', value=np.nan)

In [ ]:
merged_df = pd.merge(left = summary_records_df, right = activity_records_df, how = 'inner', on = 'day')
merged_df

In [ ]:
merged_df.keys()

In [ ]:
merged_df

In [ ]:
%%time
merged_df['t'] = pd.to_datetime(activity_records_df['timestamp'], format='%Y-%m-%d %H:%M:%S')
traj_collection = mpd.TrajectoryCollection(merged_df,
                                           'activity_id',
                                           x='position_long',
                                           y='position_lat' ,
                                           t='t',
                                           min_length=100)

In [ ]:
latest_day = merged_df.loc[merged_df['day'] == merged_df['day'].max()]

In [ ]:
heart_rate = latest_day[['hr_avg', 'hr_min', 'hr_max', 'hr']]
heart_rate.plot.box()

In [ ]:
latest_day_traj_collection = mpd.TrajectoryCollection(latest_day,
                                           'activity_id',
                                           x='position_long',
                                           y='position_lat' ,
                                           t='t',
                                           min_length=100)

In [ ]:
latest_day_traj_collection.hvplot(title='Latest Activity', geo=True, tiles='OSM', line_width=5, c='speed', colorbar=True)

In [ ]:
latest_day_traj_collection.hvplot(title='HR', geo=True, tiles='OSM', line_width=5, c='hr', colorbar=True)

In [ ]:
latest_day_traj_collection.hvplot(title='cadence', geo=True, tiles='OSM', line_width=5, c='hr', colorbar=True)

In [ ]:
latest_day_chart = latest_day[['temperature','speed','hr','cadence', 'activity_distance', 'altitude','activity_distance', 'timestamp']]
latest_day_chart.set_index('timestamp')
latest_day_chart

In [ ]:
latest_day_chart.plot()

In [ ]:
traj_collection.hvplot(title='Garmin Activities', geo=True, tiles='OSM', line_width=5, c='speed', colorbar=True)

In [ ]:
activity_records_df['speed'].plot.hist()

In [ ]:
stop_pts = mpd.TrajectoryStopDetector(traj_collection)\
   .get_stop_points(min_duration=timedelta(seconds=200), max_diameter=100)

In [ ]:
stop_pts

In [ ]:
# stop_pts.hvplot(title='Garmin Stop detections', geo=True, tiles='OSM', line_width=5, c='speed', colorbar=True)
stop_pts.hvplot( color='red', markersize=100, geo=True, tiles='OSM')